In [2]:
!ls ../data/*/o*

../data/bpr1/origin_combine:
A B

../data/bpr2/origin_combine:
A B

../data/dic/origin_combine:
A B


In [11]:
import Augmentor
from Augmentor.Operations import Operation
p = Augmentor.Pipeline('../data/bpr1/origin_combine/A/')
import numpy as np
import cv2
import matplotlib.pyplot as plt

Initialised with 9 image(s) found.
Output directory set to ../data/bpr1/origin_combine/A/output.

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [12]:
import os
import Augmentor

from Augmentor.Operations import Operation
class my_crop(Operation):
    # Here you can accept as many custom parameters as required:
    def __init__(self, probability, height, width):
        # Call the superclass's constructor (meaning you must
        # supply a probability value):
        Operation.__init__(self, probability)
        self.h = height
        self.w = width
        # Set your custom operation's member variables here as required:
#         self.num_of_folds = num_of_folds

    # Your class must implement the perform_operation method:
    def perform_operation(self, images):
        y = np.random.randint(0, images[0].size[0]-self.h)
        x = np.random.randint(0, images[0].size[1]-self.w)
        res = []
        for im in images:
            res.append(im.crop([x,y, x+self.w, y+self.h]))         
        return res
def augment(path, zoom_min=1, zoom_max=1, sample=100, scale=None):
    patha = os.path.join(path, 'A')
    pathb = os.path.join(path, 'B')
    assert os.path.exists(patha) and os.path.exists(pathb)
    p = Augmentor.Pipeline(patha)
    #     p.set_seed(0)
    p.ground_truth(pathb)
    p.add_operation(my_crop(1.,512, 512))
    if scale is not None:
        p.scale(1, scale)
    p.zoom(probability=1, min_factor=zoom_min, max_factor=zoom_max)
    p.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
    p.flip_left_right(probability=0.5)
    p.flip_top_bottom(probability=0.5)
    p.random_distortion(probability=.6, grid_width=4, grid_height=4, magnitude=10)
    p.sample(11)
    
def find_path(p_in_A, B_paths):
    s1 = p_in_A.split('-')[-1]
    for p_in_B in B_paths:
        s2 = p_in_B.split('-')[-1]
        if s1 == s2:
            return p_in_B
    return False
def get_pair_output(path_origin):
    B_paths_ = glob(os.path.join(path+'/A', 'output')+'/_*.png')
    A_paths_ = glob(os.path.join(path+'/A', 'output')+'/A_*.png')
    B_paths = []
    A_paths = []
    for pa_ in A_paths_:
        pb = find_path(pa_, B_paths_)
        if pb:
            A_paths.append(pa_)
            B_paths.append(pb)
    return A_paths, B_paths

def show_path(p1, p2):
    im1 = cv2.imread(p1, 0)
    im2 = cv2.imread(p2)    
    mask = np.zeros([*im1.shape[:2], 3], 'uint8')
    mask[...,0] = im1
    mask[...,1:] = im2[...,1:]
    plt.figure(figsize=(20,20))
    plt.show(plt.imshow(mask))

# DAICHI

In [19]:
%%time
from glob import glob
# for path in glob('../data/*/o*') :
path = '../data/dic/origin_combine/'
print('augment: ',path)
augment(path, sample=1000, zoom_min=.8, zoom_max=1.5)

output_dir_a = os.path.join(os.path.split(path)[0], 'augment/A')
output_dir_b = os.path.join(os.path.split(path)[0], 'augment/B')
os.system('rm -r {}'.format(output_dir_a))
os.system('rm -r {}'.format(output_dir_b))
os.makedirs(output_dir_a, exist_ok=True)
os.makedirs(output_dir_b, exist_ok=True)

A_paths, B_paths = get_pair_output(path)
print(len(A_paths), len(B_paths))
for pa, pb in zip(A_paths, B_paths):
    assert pa.split('-')[-1] == pb.split('-')[-1]
    name = pa.split('-')[-1]
    new_name_a = os.path.join(output_dir_a, name)
    new_name_b = os.path.join(output_dir_b, name)
    os.rename(pa, new_name_a)
    os.rename(pb, new_name_b)
#    show_path(new_name_a, new_name_b)

Executing Pipeline:   0%|          | 0/1000 [00:00<?, ? Samples/s]

augment:  ../data/dic/origin_combine/
Initialised with 9 image(s) found.
Output directory set to ../data/dic/origin_combine/A/output.9 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x7F2AF9C982E8>: 100%|██████████| 1000/1000 [02:05<00:00,  7.94 Samples/s]


1000 1000
CPU times: user 8min 4s, sys: 9.69 s, total: 8min 13s
Wall time: 2min 6s


# BPROST 1

In [111]:
%%time
from glob import glob
# for path in glob('../data/*/o*') :
path = '../data/bpr1/origin_combine/'
print('augment: ',path)
augment(path, sample=1, zoom_min=1.1, zoom_max=1.5)

output_dir_a = os.path.join(os.path.split(path)[0], 'augment/A')
output_dir_b = os.path.join(os.path.split(path)[0], 'augment/B')
os.system('rm -r {}'.format(output_dir_a))
os.system('rm -r {}'.format(output_dir_b))
os.makedirs(output_dir_a, exist_ok=True)
os.makedirs(output_dir_b, exist_ok=True)

A_paths, B_paths = get_pair_output(path)
print(len(A_paths), len(B_paths))
for pa, pb in zip(A_paths, B_paths):
    assert pa.split('-')[-1] == pb.split('-')[-1]
    name = pa.split('-')[-1]
    new_name_a = os.path.join(output_dir_a, name)
    new_name_b = os.path.join(output_dir_b, name)
    os.rename(pa, new_name_a)
    os.rename(pb, new_name_b)    

Executing Pipeline:   0%|          | 0/1 [00:00<?, ? Samples/s]

augment:  ../data/bpr2/combine_scale_50/
Initialised with 12 image(s) found.
Output directory set to ../data/bpr2/combine_scale_50/A/output.12 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x10DA92550>: 100%|██████████| 1/1 [00:00<00:00,  2.19 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x10D2F4080>: 100%|██████████| 12/12 [00:01<00:00,  8.24 Samples/s]

14 14
CPU times: user 10.4 s, sys: 481 ms, total: 10.9 s
Wall time: 1.94 s


# BPROST 2 scale = 0.3

In [112]:
%%time
path = '../data/bpr2/combine scale_30/'
augment(path, sample=1, zoom_min=2, zoom_max=2)
output_dir_a = os.path.join(os.path.split(path)[0], 'augment/A')
output_dir_b = os.path.join(os.path.split(path)[0], 'augment/B')
os.system('rm -r {}'.format(output_dir_a))
os.system('rm -r {}'.format(output_dir_b))
os.makedirs(output_dir_a, exist_ok=True)
os.makedirs(output_dir_b, exist_ok=True)

A_paths, B_paths = get_pair_output(path)
print(len(A_paths), len(B_paths))
for pa, pb in zip(A_paths, B_paths):
    assert pa.split('-')[-1] == pb.split('-')[-1]
    name = pa.split('-')[-1]
    new_name_a = os.path.join(output_dir_a, name)
    new_name_b = os.path.join(output_dir_b, name)
    os.rename(pa, new_name_a)
    os.rename(pb, new_name_b)    

Executing Pipeline:   0%|          | 0/1 [00:00<?, ? Samples/s]

Initialised with 12 image(s) found.
Output directory set to ../data/bpr2/combine scale_30/A/output.12 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x10DA92AC8>: 100%|██████████| 1/1 [00:00<00:00,  2.29 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x10D304B38>: 100%|██████████| 12/12 [00:01<00:00, 10.62 Samples/s]

14 14
CPU times: user 8.57 s, sys: 284 ms, total: 8.86 s
Wall time: 1.59 s
